In [1]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader



The next part only works if you have a training csv file stored in a folder MNIST in your drive.

In [2]:
data = pd.read_csv('drive/MyDrive/MNIST/mnist_train.csv')

In [3]:
data = torch.tensor(data.values, dtype=torch.float32)

In [4]:
Y_training = data[:,0]
X_training = data[:,1:]/255

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
dataset = TensorDataset(X_training, Y_training)
training_sets = DataLoader(dataset, 64, True)

In [7]:
m, n = X_training.size()

In [152]:
def make_param(*shape):
  return torch.normal(0.01, 0.03, size=shape, device = device, requires_grad = True)

In [149]:
lr = 0.01

In [153]:
hidden1 = 20
hidden2 = 20


w1 = make_param(n, hidden1)
w2 = make_param(hidden1, hidden2)
w3 = make_param(hidden2, 10)

b1 = make_param(hidden1, )
b2 = make_param(hidden2, )
b3 = make_param(10, )

In [154]:
def model(x):
  l1 = x@w1 +b1
  l1 = torch.relu(l1)
  l2 = l1@w2 + b2
  l2 = torch.relu(l2)
  l3 = l2@w3 + b3
  return l3

In [118]:
loss = torch.nn.CrossEntropyLoss()

In [155]:
def update():
  global w1, w2, w3, b1, b2, b3

  with torch.no_grad():
    w1 -= lr*w1.grad
    w2 -= lr*w2.grad
    w3 -= lr*w3.grad
    b1 -= lr*b1.grad
    b2 -= lr*b2.grad
    b3 -= lr*b3.grad

  w1.grad.zero_()
  w2.grad.zero_()
  w3.grad.zero_()
  b1.grad.zero_()
  b2.grad.zero_()
  b3.grad.zero_()

In [3]:
for i in range(10):
  for img_data, label in training_sets:
    img_data = img_data.to(device)
    label = label.to(device)

    logits = model(img_data)
    l = loss(logits, label.long())
    l.backward()
    update()
  print(l.item())

In [159]:
test_data = pd.read_csv('drive/MyDrive/MNIST/mnist_test.csv')

In [160]:
test_data = torch.tensor(test_data.values, dtype=torch.float32)

In [161]:
Y_test = test_data[:, 0]
X_test = test_data[:, 1:]/255

In [162]:
X_test.size()

torch.Size([10000, 784])

In [163]:
test_data = TensorDataset(X_test, Y_test)
test_sets = DataLoader(test_data, 64, False)

In [192]:
correct = 0
total = 0
for img_data, label in test_sets:
  img_data = img_data.to(device)
  label = label.to(device)
  pred = torch.argmax(model(img_data), dim = 1)
  correct += (pred==label).sum().item()

  total += label.size(0)

print((correct/total)*100)
print(total, correct)

92.9
10000 9290
